# 1505022 [ML Offline 1]

## Import statements

In [1]:
import pandas as pd
import numpy as np
import math
import random
import sklearn
from sklearn import preprocessing

### Load datasets

In [2]:
file_name_dataset_1 = "F:/Programs C and Java/Sessional Things/Assignments-Github/ML Assignments/telco-customer-churn/WA_Fn-UseC_-Telco-Customer-Churn.csv"
# file_name_dataset = "F:/Programs C and Java/Sessional Things/Assignments-Github/ML Assignments/Offline/PreprocessedSmallDatasets/evade.csv"

# file_name_dataset_2 = ""
file_name_dataset_3 = "F:/Programs C and Java/Sessional Things/Assignments-Github/ML Assignments/creditcardfraud/creditcard.csv"



In [3]:
# file_name_dataset = file_name_dataset_3
file_name_dataset = file_name_dataset_1
print(file_name_dataset)

F:/Programs C and Java/Sessional Things/Assignments-Github/ML Assignments/telco-customer-churn/WA_Fn-UseC_-Telco-Customer-Churn.csv


In [4]:
data_frame_original = pd.read_csv(file_name_dataset) 
data_frame_original.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [5]:
data_frame_original.shape

(7043, 21)

In [ ]:
""" Syntax:
c = np.linspace(0, 1, 6)   # start, end, num-points
numpy.arange(10) ==> creates 10 values of integers from 0 to 9
np.full(shape=10, fill_value=3, dtype=np.int) ==> array([3, 3, 3, 3, 3, 3, 3, 3, 3, 3])
"""

### Entropy calculation

In [5]:
"""
    Input: examples [numpy array, WITHOUT the labels], labels/classes [numpy array format]
    Output: Entropy of THIS node
    *** epsilon_small is used for log_2 operations (log2(0) may give unwanted exceptions)
    Calculation: 
        for each label x of Labels:
            probability[x] = x/num_examples
        Entropy(node) = H(node) = - [ sum of probability[x]*log_2(probability[x]) ]
"""
def calculate_entropy(examples, labels, epsilon_small = 0.0000000000000000001): # WORKING
    labels_unique = np.unique(labels) # obtain the unique labels of the data
    # gives unique label_names, and counts for each unique label_names
    label_names, label_counts = np.unique(labels,
                                         return_counts = True) 
    label_probabilities = label_counts/sum(label_counts)
    label_log_probabilities = np.log2((label_probabilities + epsilon_small))
    label_products = label_probabilities * label_log_probabilities
#     print(len(examples), " , ", label_names , " , " , label_probabilities, " , ", label_log_probabilities)
#     print(label_products)
    entropy = -1 * sum(label_products)
    if entropy == 0.0:  # to not return -0.0
        entropy = 0.0
    return entropy

In [6]:
"""
    Finds the feature types ...
"""
# If num of unique vals are greater than 20, declare as continuous [if float/int]
def check_data_type_one(unique_vals, threshold_cnt = 20):  
    if len(unique_vals) == 0:
        return "CATEGORICAL"  # just checking
    sample_val = unique_vals[0]
    if isinstance(sample_val, str):
        return "CATEGORICAL"
    if isinstance(sample_val, float):
        return "CONTINUOUS"
    if len(unique_vals) > threshold_cnt:
        return "CONTINUOUS"
    else:
        return "CATEGORICAL"
    
def find_data_types(data_frame):
    data_type_list = []
    for feature_test in data_frame.columns.values:
        val_first = data_frame[feature_test]
#         print(feature_test, " : ", val_first, "  :  ", type(val_first))
        unique_vals = np.unique(data_frame[feature_test])
#         print(unique_vals)
        type_val = check_data_type_one(unique_vals)
        data_type_list.append(type_val)
        # Keep looping
        
    return data_type_list

### Information Gain Calculation

###### Information Gain Calculation wrt one feature_column

In [19]:
"""
    Input: X (examples), Y (labels), feature_column(idx on X), feature_type [CONTINUOUS/CATEGORICAL],
            use_custom_columns {optional}, custom_columns_list {optional/=}
    Output: Information Gain wrt that feature [CATEGORICAL/BINARIZED]
            Dictionary of information gains wrt each values of that feature [CONTINUOUS]
    Dependency: Uses calculate_entropy() function written above
    Calculation:
        for each value v of examples[feature_column]:
            Partition new_examples[v] by choosing that feature.val == v [if categorical]
            Partition new_examples[v] by choosing that feature.val <= v [if continuous]
            calculate entropy of new_examples, H(S_feature_val_v)
            calculate num_examples(S_feature_val_v)/num_examples(parent_node)
            Use formula IG = H(S{parent}) - [Sum of |S_val|/|S| * H(S_val)]
    Treat either as continuous, or categorical [binarized data is treated as categorical]
"""
def calculate_information_gain(X, Y, feature_column, feature_type,
                              use_custom_columns = False, custom_columns_list = None):  # WORKING
    entropy_parent_node = calculate_entropy(X, Y) # entropy of parent node
    if ((use_custom_columns == True)) :
        unique_vals_features = custom_columns_list
    else:
        unique_vals_features = np.unique(X[:, feature_column])
#         print("In calculate_info_gain() ... feature_col = ", feature_column, " , printing unique values of the feature : ", unique_vals_features)
#         print("-->>DOING FOR ALL UNIQUE CONTINUOUS COLUMNS, feature-col = ", feature_column,
#             " , feature_type = ", feature_type, " len_unique_feature_vals = ", len(unique_vals_features))
    if feature_type == "CONTINUOUS":
#         print("-->>FEATURE_COL = ", feature_column, " .. inside if feature type == CONTINUOUS")
        # Partition into two sets ... x <= val and x > val
        info_gain_dict = {}
        for val in unique_vals_features:
            idx_left_bool = X[:, feature_column] < float(val)
            idx_right_bool = X[:, feature_column] >= float(val)
            data_left = X[idx_left_bool]
            label_left = Y[idx_left_bool]
            data_right = X[idx_right_bool]
            label_right = Y[idx_right_bool]
            entropy_left = calculate_entropy(data_left, label_left)
            entropy_right = calculate_entropy(data_right, label_right)
            info_gain = entropy_parent_node - (
                ((len(data_left)/len(X)) * entropy_left) + 
                ((len(data_right)/len(X)) * entropy_right)
            )
            info_gain_dict[val] = info_gain
#         print("+++-->>Inside calculate_info_gain() [CONTINUOUS] ... dictionary-len = ", 
#               len(info_gain_dict), " ... printing info_gain_dict .. ", info_gain_dict)
        return info_gain_dict
    else:# CATEGORICAL
        ## Partition into FOR EACH FEATURE
#         entropy_per_val = {} # empty dictionary
        num_examples_parent = len(X)
        cumulative_entropy = 0.0 # cumulative entropy for all features
        if num_examples_parent == 0: # SOMEHOW comes down to this
            print("-->>Inside calculateInfoGain() .. num_examples_parent = ", num_examples_parent,
                 " returning 0")
            return 0
        for val in unique_vals_features:
            idx_equal_to_feature = X[:, feature_column] == val
            data_of_feature = X[idx_equal_to_feature]
            label_of_feature = Y[idx_equal_to_feature]
            entropy_of_feature = calculate_entropy(data_of_feature, label_of_feature)
#             print("val = ", val, " , entropy of feature = ", entropy_of_feature)
            proportion_of_examples_in_feature = float(len(data_of_feature)) / float(num_examples_parent)
#             print("proportion of examples in feature = ", proportion_of_examples_in_feature)
            cumulative_entropy = cumulative_entropy + (proportion_of_examples_in_feature * entropy_of_feature)
#             print("cumulative entropy = ", cumulative_entropy, " parent entropy = ", entropy_parent_node)
        #             entropy_per_val[val] = entropy_of_feature
        # now subtract from parent's entropy to return the information gain
        info_gain = entropy_parent_node - cumulative_entropy
        return info_gain

## Preprocessing datasets
#### Make the last column as 'Label' and rename it to 'Label'
#### Make any string/object datatype to integers [encoding]

###### Preprocess dataset 1 -> churn dataset
###### 1. Tenure is continuous
###### 2. MonthlyCharges is continuous
###### 3. TotalCharges is continuous [object type ... changed to float type] [Some missing values ... spaces, delete those rows]
###### 4. Drop customerID column
Syntax: df.drop(df[df.score < 50].index, inplace=True)

In [13]:
def preprocess_dataset_1(df):    
    df_copy = df.copy(deep = True)
    df_copy.rename(columns = {'Churn' : 'Label'}, inplace=True)
    df_copy = df_copy.drop("customerID", axis = 1)  # drop customer ID
    df_copy.drop(df_copy[df_copy.TotalCharges == ' '].index, inplace=True)  # delete rows with spaces
    df_copy["TotalCharges"] = df_copy["TotalCharges"].astype(float)
    return df_copy

###### Dataset 3: All are continuous values, so we will binarize them
###### Dataset 3: Drop the 'time' column, keep only 20,000 NO/False labels and keep ALL YES/True labels

In [14]:
def preprocess_dataset_3(df, data_size = 20000):
    df.rename(columns={'Class':'Label'}, inplace=True)  
    # drop 'Time' column
    df = df.drop("Time", axis = 1)
    
    df_yes = df[df['Label'] == 1]
    df_no  = df[df['Label'] == 0]
    
    indices = df_no.index.tolist()
    test_indices = random.sample(population=indices, k=data_size)  # only keeps 'k' amount of data
    df_no_kept = df.loc[test_indices]
#     df_no_dropped = df.drop(test_indices)
    # recombine the 'YES' and 'NO' samples together into a new dataframe
    df = pd.concat([df_yes, df_no_kept])
    return df

###### Make the 'Label' column as separate Labels and use other columns

In [15]:
# data_frame = preprocess_dataset_3(data_frame_original)
# print(data_frame.head(5))

data_frame = preprocess_dataset_1(data_frame_original)
print(data_frame.head(5))


   gender  SeniorCitizen Partner Dependents  tenure PhoneService  \
0  Female              0     Yes         No       1           No   
1    Male              0      No         No      34          Yes   
2    Male              0      No         No       2          Yes   
3    Male              0      No         No      45           No   
4  Female              0      No         No       2          Yes   

      MultipleLines InternetService OnlineSecurity OnlineBackup  \
0  No phone service             DSL             No          Yes   
1                No             DSL            Yes           No   
2                No             DSL            Yes          Yes   
3  No phone service             DSL            Yes           No   
4                No     Fiber optic             No           No   

  DeviceProtection TechSupport StreamingTV StreamingMovies        Contract  \
0               No          No          No              No  Month-to-month   
1              Yes          No  

In [20]:
"""
    Input: dataframe
    Output: X, Y [numpy format]
"""
def separate_labels_and_features(df):
    X = df.drop("Label", axis = 1)
    Y = df["Label"]
    return X.values, Y.values

In [21]:
X, Y = separate_labels_and_features(data_frame)
print(X.shape)
print(Y.shape)
print(X)

(7032, 19)
(7032,)
[['Female' 0 'Yes' ... 'Electronic check' 29.85 29.85]
 ['Male' 0 'No' ... 'Mailed check' 56.95 1889.5]
 ['Male' 0 'No' ... 'Mailed check' 53.85 108.15]
 ...
 ['Female' 0 'Yes' ... 'Electronic check' 29.6 346.45]
 ['Male' 1 'Yes' ... 'Mailed check' 74.4 306.6]
 ['Male' 0 'No' ... 'Bank transfer (automatic)' 105.65 6844.5]]


In [22]:
print(X.shape, " ", len(X))

(7032, 19)   7032


#### All functions below are to binarize the continuous values of the dataset

In [23]:
"""
    Input: Dataset (X, Y), feature-column
    Output: Best Information Gain wrt THIS feature column
    Dependency: Uses calculate_information_gain(X, Y, feature_column, feature_type) function
                which returns the info_gain [if categorical]
                which returns dictionary of {key = split_val, value = info_gain}
"""
def get_best_IG_val_of_this_feature(X_train, Y_train, feature_col, custom_col_list):
#     print(X_train[:, 0])
    dict_info_gain_col = calculate_information_gain(X_train, Y_train, 
                                      feature_col, feature_type="CONTINUOUS",
                                                   use_custom_columns=False)
    print("-->Inside gt_best_IG_val_of_this_feature() .. col = ", feature_col ,
          " .. dictionary len = ", len(dict_info_gain_col))
#     print("Printing dictionary ... ", dict_info_gain_col)
#     dict_info_gain_col = calculate_information_gain(X, Y, feature_col, feature_type="CONTINUOUS",
#                               use_custom_columns = True, custom_columns_list = custom_col_list)  # WORKING
    # https://stackoverflow.com/questions/268272/getting-key-with-maximum-value-in-dictionary
    v = list(dict_info_gain_col.values())
    k = list(dict_info_gain_col.keys())
    return k[v.index(max(v))], max(v)
#     return dict_info_gain_col_0

In [24]:
"""
    To binarize the data, instead of comparing values for EACH continuous value,
    we will divide into 100 (or user defined number of) data points, 
    and compare with each of those values. [To make it time-efficient (Takes ~ 2mins)]
"""
def get_dictionary_of_best_split_values_for_each_col(X, Y, num_values_of_custom_cols = 100,
                        use_custom_col_list = False, custom_cols = None):
    _, ncol = X.shape
    num_cols_to_do = np.arange(ncol)
    if use_custom_col_list == True:
        num_cols_to_do = custom_cols
    dict_best_ig_feature_split_values = {}  # Dictionary to store {feature_col: split_val, max_IG} that has max IG wrt that feature column
#     for col_feature in range(ncol):  # for each feature_column
    for col_feature in num_cols_to_do:
        unique_vals_of_this_feature = np.unique(X[:, col_feature])
        left_range = int(np.ceil(min(unique_vals_of_this_feature)))
        right_range = int(np.floor(max(unique_vals_of_this_feature))) 
        cols_custom = np.linspace(left_range, right_range, num_values_of_custom_cols)
        split_val_for_max_IG, max_IG = get_best_IG_val_of_this_feature(X, Y, col_feature, cols_custom)
        dict_best_ig_feature_split_values[col_feature] = split_val_for_max_IG, max_IG
    return dict_best_ig_feature_split_values

###### Binarizes the dataset per column wrt one feature_column

In [25]:
"""
    Actually binarizes the Data wrt the above dictionary found.
"""
def binarize_dataset(X, dict_split_values_and_max_IG_per_col):
#     v = list(dict_split_values_and_max_IG_per_col.values())
    X_bin = X
    keys_list = list(dict_split_values_and_max_IG_per_col.keys())
    features_column = {}
#     print(keys_list)
    for col in keys_list:
        # dictionary[col][0] gives split-value and dictionary[col][1] gives max ig
        split_value_of_col = dict_split_values_and_max_IG_per_col[col][0]
        binarized_data_this_col = X[:, col] < split_value_of_col
        X_bin[:, col] = binarized_data_this_col
        features_column[col] = "BINARIZED"
    
    return X_bin, features_column

In [26]:
def get_new_features_list(features_list_binarized, features_list_previous_arr):
    features_list_new = {}
    for itr in range(len(features_list_previous_arr)):
        features_list_new[itr] = features_list_previous_arr[itr]
    for key in list(features_list_binarized.keys()):
        features_list_new[key] = features_list_binarized[key]
    return list(features_list_new.values()) # returns as list

In [27]:
"""
    Label encoding of labels/classes ...
"""
def label_encode_labels(Y):
    le = preprocessing.LabelEncoder()
    le.fit(Y)
    Y= le.transform(Y)
    return Y 

In [28]:
"""
    Label encoding .... 
"""
def label_encode_data(X, feature_types):
    le = preprocessing.LabelEncoder()
    for i in range(len(feature_types)):
        if feature_types[i] == "CATEGORICAL":
            le.fit(X[:, i])
            X[:, i] = le.transform(X[:, i])
    return X

In [29]:
df_2 = data_frame.drop(data_frame.columns.values[-1], axis=1)
# print(df_2.head(2))
feature_types_before = find_data_types(df_2)
print(feature_types_before, "\n", len(feature_types_before))
indices_continous = []
for i in range(len(feature_types_before)):
    if feature_types_before[i] == "CONTINUOUS":
        indices_continous.append(i)
print(indices_continous)
X, Y = separate_labels_and_features(data_frame)
print("================ BEFORE ======================")
dict_test = get_dictionary_of_best_split_values_for_each_col(X, Y, 1000,
                                                use_custom_col_list=True, custom_cols=indices_continous)
print(dict_test)
print("-------------------- AFTER -------------------------")
X_binarized, features_col_new = binarize_dataset(X, dict_test)
features_col_new = get_new_features_list(features_col_new, feature_types_before)
print(features_col_new)
X_binarized = label_encode_data(X_binarized, features_col_new)
Y = label_encode_labels(Y)
# print(X_binarized)
# Pass this binarized data into the engine

['CATEGORICAL', 'CATEGORICAL', 'CATEGORICAL', 'CATEGORICAL', 'CONTINUOUS', 'CATEGORICAL', 'CATEGORICAL', 'CATEGORICAL', 'CATEGORICAL', 'CATEGORICAL', 'CATEGORICAL', 'CATEGORICAL', 'CATEGORICAL', 'CATEGORICAL', 'CATEGORICAL', 'CATEGORICAL', 'CATEGORICAL', 'CONTINUOUS', 'CONTINUOUS'] 
 19
[4, 17, 18]
================ BEFORE ======================
-->Inside gt_best_IG_val_of_this_feature() .. col =  4  .. dictionary len =  72
-->Inside gt_best_IG_val_of_this_feature() .. col =  17  .. dictionary len =  1584
-->Inside gt_best_IG_val_of_this_feature() .. col =  18  .. dictionary len =  6530
{4: (18, 0.07579675305132505), 17: (29.4, 0.03824153343031822), 18: (348.15, 0.034475641654172384)}
-------------------- AFTER -------------------------
['CATEGORICAL', 'CATEGORICAL', 'CATEGORICAL', 'CATEGORICAL', 'BINARIZED', 'CATEGORICAL', 'CATEGORICAL', 'CATEGORICAL', 'CATEGORICAL', 'CATEGORICAL', 'CATEGORICAL', 'CATEGORICAL', 'CATEGORICAL', 'CATEGORICAL', 'CATEGORICAL', 'CATEGORICAL', 'CATEGORICAL', 

In [30]:
# Y = label_encode_labels(Y)
print(X_binarized)
print(Y)

[[0 0 1 ... 2 False True]
 [1 0 0 ... 3 False False]
 [1 0 0 ... 3 False True]
 ...
 [0 0 1 ... 2 False True]
 [1 1 1 ... 3 False True]
 [1 0 0 ... 0 False False]]
[0 0 1 ... 0 1 0]


In [80]:
print(data_frame.columns[4], " ", data_frame.columns[17], " ", data_frame.columns[18])

tenure   MonthlyCharges   TotalCharges


In [31]:
for key in list(dict_test.keys()):
    print("Col = ", key , " :-> Split-Val = ", dict_test[key][0], " , InfoGain = ", dict_test[key][1])

Col =  4  :-> Split-Val =  18  , InfoGain =  0.07579675305132505
Col =  17  :-> Split-Val =  29.4  , InfoGain =  0.03824153343031822
Col =  18  :-> Split-Val =  348.15  , InfoGain =  0.034475641654172384


In [32]:
"""
    Compute new feature types ... [Binarized/Categorical/Continuous]
"""
def obtain_new_feature_types(feature_types_prev, dict_feature_types):  # WORKING
    keys = list(dict_feature_types.keys())
    feature_types_latest = feature_types_prev # Copy previous feature types ...
    for i in keys:
        feature_types_latest[i] = dict_feature_types[i]
    return feature_types_latest

In [33]:
# feature_types = obtain_new_feature_types(feature_types_before, features_col_new)
# print(feature_types, " ", len(feature_types))
feature_types = features_col_new
print(feature_types)

['CATEGORICAL', 'CATEGORICAL', 'CATEGORICAL', 'CATEGORICAL', 'BINARIZED', 'CATEGORICAL', 'CATEGORICAL', 'CATEGORICAL', 'CATEGORICAL', 'CATEGORICAL', 'CATEGORICAL', 'CATEGORICAL', 'CATEGORICAL', 'CATEGORICAL', 'CATEGORICAL', 'CATEGORICAL', 'CATEGORICAL', 'BINARIZED', 'BINARIZED']


### Convert to numpy and train_test_split using scikitlearn

In [34]:
from sklearn.model_selection import train_test_split
from pprint import pprint
from sklearn.metrics import confusion_matrix

In [76]:
# X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=7)
X_train, X_test, Y_train, Y_test = train_test_split(X_binarized, Y, test_size=0.2, random_state=0)
# X_train, X_test, Y_train, Y_test = train_test_split(X_binarized, Y, test_size=0.2)

In [77]:
print(X_train.shape, " ", Y_train.shape , " ", X_test.shape, " ", Y_test.shape)

(5625, 19)   (5625,)   (1407, 19)   (1407,)


In [78]:
print(X_train[0][0] == 0)
print((X_train.shape[1]))
for i in range((X_train.shape[1])):
#     print(i, end=' ')
    print("-----------------------------------------------")
    print(np.unique(X_train[:, i], return_counts = True))
    print(np.unique(X_test[:, i], return_counts = True))

False
19
-----------------------------------------------
(array([0, 1], dtype=object), array([2798, 2827], dtype=int64))
(array([0, 1], dtype=object), array([685, 722], dtype=int64))
-----------------------------------------------
(array([0, 1], dtype=object), array([4722,  903], dtype=int64))
(array([0, 1], dtype=object), array([1168,  239], dtype=int64))
-----------------------------------------------
(array([0, 1], dtype=object), array([2920, 2705], dtype=int64))
(array([0, 1], dtype=object), array([719, 688], dtype=int64))
-----------------------------------------------
(array([0, 1], dtype=object), array([3935, 1690], dtype=int64))
(array([0, 1], dtype=object), array([998, 409], dtype=int64))
-----------------------------------------------
(array([False, True], dtype=object), array([3524, 2101], dtype=int64))
(array([False, True], dtype=object), array([882, 525], dtype=int64))
-----------------------------------------------
(array([0, 1], dtype=object), array([ 550, 5075], dtype=i

### Form decision tree bulding algorithm

In [38]:
"""
    Returns the majority of the label
"""
def return_majority_label(labels):
    # obtains unique labels AND also the counts of those unique labels
    label_names, label_counts = np.unique(labels, return_counts = True)
    index_max = label_counts.argmax()
    labels_with_max_count = label_names[index_max]
    return labels_with_max_count

In [289]:
"""
    Class DecisionTreeNode to store the decision trees/subtrees nodes.
    CAN'T HAVE CONTINUOUS DATA [ONLY BINARIZED/CATEGORICAL]
"""
class DTreeNode:
    def __init__(self):
        # self.sub_trees = []  # List of subtrees/children
        # self.feature_values = [] # list of feature values of children [To ask question use indices]
        self.children = {} # [feature_val: DTreeNode]
        self.feature_col = -1 # which feature column THIS node contains as a question
        self.is_leaf_node = False # by default, shouldn't be a leaf
        self.classification = "NONE" # also consider as the plurality value
        
    def printTree(self, spaces_num = 0):
        node = self
        if node.is_leaf_node == True:  # Only print the classification column/feature
            print(" " * spaces_num, "Lab(", node.classification, ")")
            return
        else:  # Print the question
            # print(" " * spaces_num, node.feature_col, " ", node.get_comparison_mark(), " ", node.feature_val)
            print("\n", ("    " * spaces_num), "Q(", node.feature_col, ")")
        spaces_num = spaces_num + 1
        for key in list(self.children.keys()):
            print("  "*spaces_num, " == ", key)
            node = self.children[key]
            node.printTree(spaces_num)
            

In [290]:
"""
    Testing above data structure.
"""
root = DTreeNode()
root.feature_col = 0
child1 = DTreeNode()
child1.is_leaf_node = True
child1.classification = 0
child2 = DTreeNode()
child2.is_leaf_node = True
child2.classification = 1
child3 = DTreeNode()
child3.feature_col = 22
child31 = DTreeNode()
child31.is_leaf_node = True
child31.feature_col = 8
child31.classification = 0
child32 = DTreeNode()
child32.is_leaf_node = True
child32.feature_col = 10
child32.classification = 1
child3.children[13] = child31
child3.children[31] = child32
root.children[1] = child1
root.children[2] = child2
root.children[3] = child3
root.printTree()


  Q( 0 )
    ==  1
  Lab( 0 )
    ==  2
  Lab( 1 )
    ==  3

      Q( 22 )
      ==  13
   Lab( 0 )
      ==  31
   Lab( 1 )


In [300]:
print(return_majority_label([0, 0, 0, 1]))
print(return_majority_label([0]))
a = b = 1
print(a, " ", b)

dicti = {1:"A", 2:"B", 3:"C", 4:"D"}
print(list(dicti.keys())[0])
# print(len(dicti.where("A")))

0
0
1   1
1


In [350]:
"""
    Can't have CONTINUOUS data [Will be binarized first]
    Decision Tree Classifier
"""    
class DTreeClassifier:
    def __init__(self):
        self.d_tree_root = DTreeNode()
        self.max_depth = 5
    
    def form_a_leaf_node(self, labels):
        leaf_node = DTreeNode()
        leaf_node.is_leaf_node = True
        leaf_node.classification = return_majority_label(labels)
        return leaf_node
    ##### Can't have CONTINUOUS data [ONLY CATEGORICAL/BINARIZED DATA IS ALLOWED !!]
    def recursive_fit(self, X, Y, current_depth, max_depth):
        ## 1. If current-depth == max-depth [Base Case]
        if current_depth == max_depth:
            return self.form_a_leaf_node(Y) # return the leaf node with majority of the current labels
        ## 2. If EXACTLY one label ... return that
        if (len(np.unique(Y)) == 1):
            return self.form_a_leaf_node(Y) # return the leaf node with majority of the current labels
        current_depth = current_depth + 1  # increment current_depth variable
        val_max_IG = col_max_IG = -1
        for col in range(0, X.shape[1]):  ## for each number of columns/features
            ig_this_col = calculate_information_gain(X, Y, col, "CATEGORICAL")
#             print("col = ", col, " ig_col = ", ig_current_col)
            if ig_this_col > val_max_IG:
                val_max_IG = ig_this_col
                col_max_IG = col
        
        ## 3. Max IG obtained is 0 [no more examples/features]
        if val_max_IG == 0:
            return self.form_a_leaf_node(Y)
        
        d_tree_node = DTreeNode()
        d_tree_node.classification = return_majority_label(Y)  ### Saves the pluarility value of THIS node [used in prediction]
        ## -> Recursion
        for val_of_this_feature in np.unique(X[:, col_max_IG]):
            index_child = (X[:, col_max_IG] == val_of_this_feature)
            X_child = X[index_child]
            Y_child = Y[index_child]

            d_tree_node.is_leaf_node = False  ### is an internal node
            d_tree_node.feature_col = col_max_IG  ### question is to be asked on this column/feature
            # d_tree_node.feature_values.append(val_of_this_feature)
            if (X_child.shape[0] == 0): # no more examples [DON'T return as other values of list will be empty]
                d_tree_node.children.children[val_of_this_feature] = self.form_a_leaf_node(Y)  # return parent's max plurality value
                # d_tree_node.sub_trees.append(self.form_a_leaf_node(labels))
            else:
            ##### DICTIONARY {key = feature_val, value = sub-tree}
                sub_tree = self.recursive_fit(X_child, Y_child, current_depth, max_depth)
                d_tree_node.children[val_of_this_feature] = sub_tree
                # d_tree_node.sub_trees.append(sub_tree)
        ### END OF FOR LOOP, return d_tree_node
        return d_tree_node
    
    def printTree(self):
        print("-->Inside printTree")
        self.d_tree_root.printTree()

    def fit(self, examples, labels, max_depth = 5):
        dt_node = self.recursive_fit(examples, labels, current_depth = 0, max_depth=max_depth)
        self.d_tree_root = dt_node
        print("Fit done for, max-depth = ", max_depth)
        
    def predict_one_example(self, example_to_predict):
        self.d_tree_root_backup = self.d_tree_root
        root = self.d_tree_root
        if root == None:
            raise Exception('Root of the Decision Tree is null !! [In predict()]')
        while root is not None:
            if root.is_leaf_node == True: # classify ... since, it is the leaf
                self.d_tree_root = self.d_tree_root_backup
                return root.classification
            else:
                col_to_process = root.feature_col
                val_present_in_example = example_to_predict[col_to_process]
                bool_found = False
#                 print("In col = ", col_to_process, " len root.children = ", len(root.children))
                # for idx in range(len(root.feature_values)):
                keys_feature_vals = list(root.children.keys())
                for feature_val in keys_feature_vals:
                    if example_to_predict[root.feature_col] == feature_val:
                        root = root.children[feature_val]
                        bool_found = True
                        break
                    
                if bool_found == False: ### SHOULD RETURN PARENT's PLURALITY VALUE
                    self.d_tree_root = self.d_tree_root_backup
                    return root.classification # PARENT's plurality value
                    #raise Exception("The value of ", val_present_in_example, " doesn't exist for col_idx = ", col_to_process)
                
    def predict(self, examples_test):
        labels = []
        for example in examples_test:
            yp1 = self.predict_one_example(example)
            labels.append(yp1)
        return labels
    
    
    def print_metrics(self, Y_true, Y_predicted):
        # print(len(Y_pred))
        TN, FP, FN, TP = confusion_matrix(Y_true, Y_predicted).ravel()
        accuracy = (TP + TN)/(TP+TN+FP+FN)
        recall = (TP)/(TP + FN)
        specificity = (TN)/(TN + FP)
        precision = (TP)/(TP + FP)
        false_discovery_rate = (FP)/(TP + FP)
        f1_score = 2*((precision * recall) / (precision + recall))
        print("TN = ", TN, " FP = ", FP, " FN = ", FN, " TP = ", TP)
        print("Accuracy = ", accuracy*100, "%")
        print("TPR = Sensitivity = Recall = ", recall*100, "%")
        print("Specificity = ", specificity*100, "%")
        print("Precision = ", precision*100, "%")
        print("FDR = False Discovery Rate = ", false_discovery_rate*100, "%")
        print("F1 Score = ", f1_score*100, "%")
        # return TN, FP, FN, TP, accuracy, recall, specificity, precision, false_discovery_rate, f1_score

In [351]:
d_tree = DTreeClassifier()
# d_tree.fit(X_train[0:100], Y_train[0:100], max_depth = 10000)
d_tree.fit(X_train, Y_train, max_depth = 10000)

Fit done for, max-depth =  10000


### Prediction Script

In [352]:
Y_pred_train = d_tree.predict(X_train)
print("------------- Training ----------------")
d_tree.print_metrics(Y_train, Y_pred_train)

------------- Training ----------------
TN =  4060  FP =  65  FN =  240  TP =  1260
Accuracy =  94.57777777777778 %
TPR = Sensitivity = Recall =  84.0 %
Specificity =  98.42424242424242 %
Precision =  95.09433962264151 %
FDR = False Discovery Rate =  4.905660377358491 %
F1 Score =  89.20353982300884 %


In [353]:
Y_pred_test = d_tree.predict(X_test)
print("------------- Testing ----------------")
d_tree.print_metrics(Y_test, Y_pred_test)

------------- Testing ----------------
TN =  881  FP =  157  FN =  205  TP =  164
Accuracy =  74.27149964463398 %
TPR = Sensitivity = Recall =  44.44444444444444 %
Specificity =  84.8747591522158 %
Precision =  51.09034267912772 %
FDR = False Discovery Rate =  48.90965732087228 %
F1 Score =  47.53623188405797 %


In [111]:
print(sklearn.metrics.classification_report(Y_test, Y_pred))

              precision    recall  f1-score   support

           0       0.74      0.95      0.83      1038
           1       0.35      0.08      0.14       369

    accuracy                           0.72      1407
   macro avg       0.55      0.51      0.48      1407
weighted avg       0.64      0.72      0.65      1407



In [354]:
from sklearn.tree import DecisionTreeClassifier
dtc = DecisionTreeClassifier(criterion='entropy')
dtc.fit(X_train, Y_train)

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [355]:
Y_p_train = dtc.predict(X_train)
Y_p_test = dtc.predict(X_test)
print("=============== TRAINING =================")
d_tree.print_metrics(Y_p_train, Y_train)
print("+++++++++++++++ TESTING ++++++++++++++++++")
d_tree.print_metrics(Y_p_test, Y_test)

=============== TRAINING =================
TN =  4060  FP =  238  FN =  65  TP =  1262
Accuracy =  94.61333333333334 %
TPR = Sensitivity = Recall =  95.10173323285606 %
Specificity =  94.46254071661238 %
Precision =  84.13333333333334 %
FDR = False Discovery Rate =  15.866666666666667 %
F1 Score =  89.28192430137956 %
+++++++++++++++ TESTING ++++++++++++++++++
TN =  867  FP =  196  FN =  171  TP =  173
Accuracy =  73.91613361762616 %
TPR = Sensitivity = Recall =  50.2906976744186 %
Specificity =  81.56161806208843 %
Precision =  46.883468834688344 %
FDR = False Discovery Rate =  53.116531165311656 %
F1 Score =  48.527349228611506 %


In [107]:
Y_pred_Test = dtc.predict(X_test)
val = 1
TN, FP, FN, TP, accuracy, recall, specificity, precision, false_discovery_rate, f1_score = calculate_metrics(val, X_test, Y_test, give_y_pred=True, y_pred_given=Y_pred_Test)
print("------------------ Testing --------------------------")
print("TN = ", TN, " FP = ", FP, " FN = ", FN, " TP = ", TP)
print("Accuracy = ", accuracy*100, "%")
print("TPR = Sensitivity = Recall = ", recall*100, "%")
print("Specificity = ", specificity*100, "%")
print("Precision = ", precision*100, "%")
print("FDR = False Discovery Rate = ", false_discovery_rate*100, "%")
print("F1 Score = ", f1_score*100, "%")

1407
------------------ Testing --------------------------
TN =  860  FP =  178  FN =  193  TP =  176
Accuracy =  73.6318407960199 %
TPR = Sensitivity = Recall =  47.696476964769644 %
Specificity =  82.85163776493256 %
Precision =  49.717514124293785 %
FDR = False Discovery Rate =  50.282485875706215 %
F1 Score =  48.686030428769016 %


In [ ]:
def AdaBoost():
    pass

In [107]:
a = np.array([1, 2, 3])
b = np.array([2, 4, 6])
print(a, " ", b)
c = np.multiply(a, b)
print(c)

[1 2 3]   [2 4 6]
[ 2  8 18]
